In [3]:
import gradio as gr
import pickle
from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
with open('final_model_XGBoost.pkl','rb') as file:
    model = pickle.load(file)

In [5]:
model.feature_names_in_

array(['lead_time', 'market_segment_type', 'no_of_special_requests',
       'avg_price_per_room', 'no_of_adults', 'no_of_weekend_nights',
       'required_car_parking_space', 'no_of_week_nights', 'arrival_Day',
       'arrival_Month', 'arrival_Weekday'], dtype='<U26')

In [6]:
model.predict_proba([[20,1,0,100,2,2,0,1,4,5,2]])[:,1][0]

0.46858412

In [7]:
def prediction(lt,mst,spcl,price,adul,wkend,park,wk,ar_d,ar_m,ar_w):
    input_Data = [[lt,mst,spcl,price,adul,wkend,park,wk,ar_d,ar_m,ar_w]]
    pred = model.predict_proba(input_Data)[:,1][0]
    if pred>0.5:
        return f'This booking is more likely to canceled: chances = {round(pred*100,2)}%'
    else:
        return f'This booking is less likely to get canceled: chances = {round(pred*100,2)}%'

In [8]:
 prediction(10,1,1,150,2,2,0,1,4,5,3)

'This booking is less likely to get canceled: chances = 23.2%'

In [9]:
iface = gr.Interface(fn=prediction,inputs=[gr.Number(label='How many prior days booking was made?'),
                                          gr.Dropdown([('Online',1),('Offline',0)],label='How the booking was made?'),
                                          gr.Dropdown([0,1,2,3,4,5],label='How many special requests were made?'),
                                          gr.Number(label='What is the price per room offered?'),
                                          gr.Dropdown([1,2,3,4],label='How many adults per room?'),
                                          gr.Number(label='How many weekend nights in the stay?'),
                                          gr.Dropdown([('Yes',1),('No',0)],label='Does booking includes parking facility?'),
                                          gr.Number(label='How many week nights in the stay?'),
                                          gr.Slider(minimum=1,maximum=31,step=1,label='What is the day of arrival?'),
                                          gr.Slider(minimum=1,maximum=12,step=1,label='What is month of arrival?'),
                                          gr.Dropdown([('Monday',0),('Tuesday',1),('Wednesday',2),('Thursday',3),('Friday',4),('Saturday',5),('Sunday',6)],
                                                      label='What is weekday of arrival?')],
                     outputs=gr.Textbox(label='Prediction'),
                    title='INN Group of Hotels',
                    description='This application will forecast the cancellation of booking',
                    allow_flagging='never')

In [10]:
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
